## 1. 导入相关包



In [29]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

## 2. 加载数据集

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
datasets = load_dataset("csv", data_files="/content/drive/MyDrive/ChnSentiCorp_htl_all.csv", split="train")
datasets = datasets.filter(lambda x: x["review"] is not None)
datasets

Dataset({
    features: ['label', 'review'],
    num_rows: 7765
})

## 3. 划分数据集

In [32]:
datasets = datasets.train_test_split(test_size=0.1)
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 777
    })
})

In [33]:
for i in range(10):
    print(trainset[i])

{'label': 0, 'review': '关于携程：携程在如家的价格谈判能力值得怀疑，这次订的是套房，通过携程订价格289，网上未显示门市价格。到了如家局前店后发现门市价为299元，携程价格仅优惠10元。再后来常州的朋友（就住在如家旁边）告诉我，如果直接预定，能轻松打到240-250的价格折扣。这样我就非常怀疑携程的价格谈判能力了，通过携程预定是为了经济，但拿到的价格之比公开的门市价低10元，根本不具备价格优势，甚至通过当地的朋友都能拿到比携程更便宜的价格，那我们还用携程干什么？关于酒店：套房设施太为陈旧，实在与289的价格不符，尤其是一楼客厅的沙发，简直是房间的最大败笔，灰绿色的沙发，实在很像从垃圾堆里检出来的。套房在1层，是上下层的复式结构，上层无窗户，下层有窗户，窗户外1米是一堵高墙，可以依稀透进来一点光线，感觉就是地下室。睡觉前在屋子里拍蚊子，拍死不下20只。住如家住了不少店，这样的房间还是第一次遭遇，出游不想影响心情，所以回来后再投诉，希望如家和携程能有所反馈。'}
{'label': 1, 'review': '优点：房间条件简洁、气氛温馨，住宿环境比较安静，价格比较合理，服务态度好；不足：窗户几乎与后面的房顶等高，加上没有防盗窗（网），安全感不高。'}
{'label': 1, 'review': '服务很好，设施也蛮不错的，但房间隔音实在太差了。宾馆反馈2007年12月28日：首先非常感谢您选择东方大酒店,也非常感谢您给我们提出了宝贵的意见,酒店近期楼顶正在加层建造西餐厅,施工初期会有一些噪音工段,同时我们对工程施工时间有明确的规定,正常休息时间内是不允许施工的;再次对您表示诚挚的歉意,东方大酒店在不久后又将会以全新的形象迎接您的再次光临!'}
{'label': 0, 'review': '挂的是三星的牌牌，但室内、用具、环境及性价比还是差太多，现在可能是哈市的旅游旺季，因为是冰雪的季节吗？房间内有股汗味让人实在受不了！'}
{'label': 0, 'review': '看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但没有可以吃的东东，环境是好但天气太冷，总之我以后不会再住那里。'}
{'label': 0, 'review': '预定中午12点入住，中午11点半左右到，告知房间正在清理。心想，清

## 4. 数据集预处理(如果使用Trianer就不再需要Dataloader了)

In [35]:

import torch
tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def process_function(examples):
    tokenized_examples = tokenizer(examples["review"], max_length=128, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

Map:   0%|          | 0/6988 [00:00<?, ? examples/s]

Map:   0%|          | 0/777 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 777
    })
})

Hugging Face 的 Trainer 或很多自定义模型都期望输入是一个包含 labels 键的字典

collate_func()返回的是：

{

  'input_ids': ...,

  'attention_mask': ...,

  'labels': tensor([1, 0, 1])
  
}

## 5. 创建模型

In [36]:

model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
model.config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "dtype": "float32",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.57.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

## 6. 创建评估函数

In [38]:
!pip install evaluate

In [48]:
import evaluate
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [40]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

In [41]:
import transformers
print(transformers.__version__)
print(transformers.__file__)

from transformers import TrainingArguments
print(TrainingArguments.__module__)

4.57.3
/usr/local/lib/python3.12/dist-packages/transformers/__init__.py
transformers.training_args


## 7. 创建TrainingArguments

In [45]:
from transformers.training_args import TrainingArguments
train_args = TrainingArguments(output_dir="./checkpoints",      # 输出文件夹
                               per_device_train_batch_size=64,  # 训练时的batch_size
                               per_device_eval_batch_size=128,  # 验证时的batch_size
                               logging_steps=10,                # log 打印的频率
                               save_strategy="epoch",           # 保存策略
                               eval_strategy = "epoch",
                               save_total_limit=3,              # 最大保存数
                               learning_rate=2e-5,              # 学习率
                               weight_decay=0.01,               # weight_decay
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True,     # 训练完成后加载最优模型
                               # 下面这三行是关键！彻底关闭 wandb
                               report_to=["none"],                  # 或者直接写 report_to=[]
                               disable_tqdm=False)                  # 保留进度条
train_args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=True,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.EPOCH,
eval_use_gather_object=False,

## 8. 创建Trainer

In [46]:
from transformers import DataCollatorWithPadding
trainer = Trainer(model=model,
                  args=train_args,
                  train_dataset=tokenized_datasets["train"],
                  eval_dataset=tokenized_datasets["test"],
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=eval_metric)

## 9. 模型训练

In [49]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.213900,0.267553,0.893179,0.923502
2,0.207200,0.286718,0.885457,0.916745
3,0.165700,0.278789,0.891892,0.922936


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=330, training_loss=0.21409363349278768, metrics={'train_runtime': 32.6999, 'train_samples_per_second': 641.103, 'train_steps_per_second': 10.092, 'total_flos': 351909933963264.0, 'train_loss': 0.21409363349278768, 'epoch': 3.0})

## 10.模型评估

In [50]:
trainer.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.2675528824329376,
 'eval_accuracy': 0.8931788931788932,
 'eval_f1': 0.9235023041474655,
 'eval_runtime': 0.5112,
 'eval_samples_per_second': 1519.806,
 'eval_steps_per_second': 13.692,
 'epoch': 3.0}

## 11. 模型预测

In [51]:
trainer.predict(tokenized_datasets["test"])

PredictionOutput(predictions=array([[ 1.1758354, -0.9686211],
       [-1.55729  ,  1.8111116],
       [-2.193197 ,  2.4891129],
       ...,
       [ 1.2666239, -1.8134947],
       [-2.2161593,  2.603417 ],
       [-2.107583 ,  2.4603333]], dtype=float32), label_ids=array([0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1,
    

In [52]:
from transformers import pipeline

id2_label = id2_label = {0: "差评！", 1: "好评！"}
model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

Device set to use cuda:0


In [53]:
sen = "我觉得不错！"
pipe(sen)

[{'label': '好评！', 'score': 0.9836521148681641}]